# News_working

In [11]:
path = '/Users/jacob/Desktop/studies/misc/kaggle/two_sigma_news/scripts'
import resource
import sys
sys.path.append(path)
from sigma_libs import *
# import backtrader as bt

df_market = pd.read_csv('../data/marketdata_sample.csv') 
df_news = pd.read_csv('../data/news_sample.csv')

In [12]:
print("Checkpoint 1")
print('Memory usage: %s (kb)' 
%resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

Checkpoint 1
Memory usage: 120127488 (kb)


In [13]:
asset_list = set(df_market.assetName)

In [14]:
df_news[df_news.assetName.isin(asset_list)]

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
62,2007-01-02 04:40:28+00:00,2007-01-02 04:40:28+00:00,2006-12-29 04:09:50+00:00,89c34671612f601d,PRESS DIGEST - Washington Post Business - Dec 29,3,2,RTRS,"{'ENT', 'HDWR', 'DPR', 'US', 'STX', 'SFWR', 'R...","{'PCU', 'PMF', 'PCO', 'DNP', 'E', 'PSC', 'ESN'...",2542,1,PRESS DIGEST,False,20,460,"{'AAPL.DE', 'AAPL.OQ', 'AAPL.F', 'AAPL.O'}",Apple Inc,4,0.182574,-1,0.813967,0.12827,0.057764,82,0,0,0,3,3,0,0,0,112,153
65,2007-01-02 05:01:02+00:00,2007-01-02 05:01:02+00:00,2007-01-02 05:01:02+00:00,8d2ebc5958ea4f1f,Adobe Appoints Former Borland Exec to Senior V...,3,1,BSW,"{'BACT', 'BUS', 'US', 'NEWR', 'MNGISS', 'LEN'}","{'CNR', 'BSW'}",2405,1,NaN,False,18,382,"{'ADBE.OQ', 'ADBE.O', 'ADBE.DE'}",Adobe Systems Inc,1,1.000000,1,0.088112,0.37209,0.539798,365,0,0,0,0,0,0,0,0,0,0


In [ ]:
set([asset for asset in df_market.assetCode if re.match(r'VIA', asset)])

df_market[df_market.assetCode == 'VIAB.O'].index[0]

df_market.query('index == 3465311')

df_market.query('index == 1336')

df_market.query('index == 1897421')

In [7]:
feat = 'universe'
summary = df_market[feat].value_counts(dropna=False)
print(summary.head())
print(len(summary))

1.0    68
0.0    32
Name: universe, dtype: int64
2


In [ ]:
aggregations = ['mean']
gp = df_news.groupby(['assetCode', 'date']).agg(aggregations)
gp.columns = pd.Index(["{}_{}".format(e[0], e[1]) for e in gp.columns.tolist()])
# Interesting way to rename columns
gp.reset_index(inplace=True)
# Set datatype to float32
float_cols = {c: 'float32' for c in gp.columns if c not in ['assetCode', 'date']}
# return gp.astype(float_cols)
df_agg = gp.astype(float_cols)

In [ ]:
date = df.date
num_target = df.returnsOpenNextMktres10.astype('float32')
bin_target = (df.returnsOpenNextMktres10 >= 0).astype('int8')
universe = df.universe.astype('int8')
# Drop columns that are not features
df.drop(['returnsOpenNextMktres10', 'date', 'universe', 'assetCode', 'assetName', 'time'], 
        axis=1, inplace=True)
df = df.astype('float32')  # Set all remaining columns to float32 datatype
gc.collect()

In [ ]:
df_market.drop_duplicates()

## Features: Subjects & Audiences

In [64]:
# SUBJECT INDICATORS
import re
df_news.subjects = df_news.subjects.apply(lambda x: re.sub(r'[n{}\']', '', x)).replace(" ", "")

df_sub = df_news[['subjects']]
# df_news.drop(['subjects'], axis=1, inplace=True)
# df_sub.head()

x = df_sub.subjects.str.split(r',', expand=True)
x = x.applymap(lambda x: x.strip() if type(x) == str else x)
x = x.stack()

In [56]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(x)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [57]:
lb.classes_



array(['AIR', 'APL', 'ASIA', 'AU', 'AUT', 'BACT', 'BD', 'BE', 'BEV', 'BG',
       'BKRT', 'BLD', 'BNK', 'BR', 'BUS', 'BY', 'CA', 'CEN', 'CH', 'CHE',
       'CN', 'CO', 'CON', 'CRU', 'DE', 'DIV', 'DPR', 'DRU', 'DRV', 'ECI',
       'ELC', 'ELG', 'ELI', 'EMRG', 'ENR', 'ENT', 'ES', 'EU', 'EUROPE',
       'FEA', 'FI', 'FILM', 'FIN', 'FR', 'FUND', 'GB', 'HDWR', 'HK',
       'HOT', 'HT', 'ID', 'IE', 'IL', 'IN', 'IND', 'INS', 'INT', 'IQ',
       'IT', 'JOB', 'JP', 'KR', 'LATAM', 'LEI', 'LEN', 'LIF', 'LOA',
       'MAC', 'MCE', 'MEAST', 'MET', 'MNGISS', 'MRG', 'MUL', 'MUNI', 'MX',
       'MY', 'NEWR', 'NEWS', 'NGS', 'NL', 'NORD', 'PLCY', 'PRESS', 'PT',
       'PUB', 'REA', 'REC', 'REG', 'REGS', 'RESF', 'RET', 'RO', 'RTRS',
       'SFWR', 'SG', 'SHP', 'STIR', 'STL', 'STX', 'TBCS', 'TEL', 'TH',
       'TRD', 'TW', 'US', 'USC', 'WASH', 'WEU', 'WHO', 'WWW', 'XREF'],
      dtype='<U6')

In [58]:
df_n = lb.transform(x)


In [63]:
# pd.DataFrame(df_n)#.groupby(level=0).sum()


In [ ]:
df_sub = pd.get_dummies(x, prefix='sub_').groupby(level=0).sum()

print(df_sub.shape)
df_sub.head()


In [ ]:
df_news.audiences.head()


In [ ]:
subjects = ['LEN', 'CMPNY', 'US', 'AMERS', 'RTRS', 'BLR', 'BACT', 'FINS', 'CYCS',
       'BISV', 'TMT', 'TECH', 'EUROP', 'RES', 'ENER', 'INDS', 'WEU', 'BSVC',
       'HECA', 'GEN', 'SHOPAL', 'BANK', 'NEWR', 'SWIT', 'INVS', 'SHOP', 'PHMR',
       'ITSE', 'OILG', 'RCH', 'NCYC', 'INVM', 'GB', 'TEEQ', 'EMRG', 'EXPL',
       'STX', 'BMAT', 'CCOS', 'ASIA', 'REP', 'RESF', 'DEAL1', 'SOFW', 'RSPC',
       'ENEQ', 'PHAG', 'HLTH', 'MRCH', 'COM']


In [ ]:
# AUDIENCE INDICATORS
import re
df_news.audiences = df_news.audiences.apply(lambda x: re.sub(r'[n{}\']', '', x)).replace(" ", "")

df_aud = df_news[['audiences']]



x = df_aud.audiences.str.split(r',', expand=True)
x = x.applymap(lambda x: x.strip() if type(x) == str else x)
x = x.stack()

df_aud = pd.get_dummies(x, prefix='aud_').groupby(level=0).sum()

print(df_aud.shape)
df_aud.head()

In [ ]:
import re
df_news.subjects = df_news.subjects.apply(lambda x: re.sub(r'[n{}\']', '', x)).replace(" ", "")

df_sub = df_news[['subjects']]
df_news.drop(['subjects'], axis=1, inplace=True)
# df_sub.head()

x = df_sub.subjects.str.split(r',', expand=True)
x = x.applymap(lambda x: x.strip() if type(x) == str else x)



In [ ]:
counts = pd.DataFrame(x.values.flatten(), columns=['subs']).subs.value_counts()
# list(counts.index[:50]) # Took the top 50

In [33]:
# import zipline
# !QUANDL_API_KEY='xTmgdiCLWxt_bx3CUFAT' zipline ingest -b quandl

In [30]:
!pip freeze| grep zip

zipline==1.3.0


In [ ]:
# STATSMODELS - PURSUIT OF THE MYSTERIOUS RESIDUAL
from statsmodels.tsa.seasonal import seasonal_decompose 

vec = vec.fillna(0)
decomposition=seasonal_decompose(vec)
